# Case Study: Topic Modelling for Amazon Review
Objective: A mobile company wants to understand the voice of the customer by analyzing the reviews of their product on Amazon and the topics that customers are talking about.
# Tasks -

1. Normalize case
2. Tokenize (using word_tokenize from NLTK)
3. POS tagging using the NLTK pos tagger
4. For the topic model, we would want to include only nouns
 - First, find out all the POS tags that correspond to nouns
5. Lemmatize (you want different forms of the terms to be treated as one, don't worry about providing POS tag to lemmatizer for now)
6. Remove stop words and punctuation
7. Create a topic model using LDA on the cleaned up data with 10 topics
 - choose the topic model parameters carefully
 - what is the coherence of the model?
8. Analyze the topics, which pairs of topics can be combined?
9. Create topic model using LDA with what you think is the optimal number of topics
 - choose the topic model parameters carefully
 - is the coherence better now?
10. The business finally needs to be able to interpret the topics
 - name each of the identified topics
 - create a table with the topic name and the top 10 terms in each to present to business

#### Some code to get you started

In [1]:
import warnings
warnings.filterwarnings("ignore")

# Importing the usual utilities
import numpy as np, pandas as pd
import re, random, os, string

from pprint import pprint #pretty print

from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

#### Reading in the reviews file

In [2]:
# Import the dataset
reviews = pd.read_csv("K8 Reviews v0.2.csv")
reviews.head()

,sentiment,review
0,1,Good but need updates and improvements
1,0,"Worst mobile i have bought ever, Battery is dr..."
2,1,when I will get my 10% cash back.... its alrea...
3,1,Good
4,0,The worst phone everThey have changed the last...


## Task 1. Normalize case

In [3]:
reviews_lower = [sent.lower() for sent in reviews.review.values]
reviews_lower[0]

'good but need updates and improvements'

## Task 2. Tokenize (using word_tokenize from NLTK)

In [4]:
import nltk
nltk.download('punkt')

reviews_token = [word_tokenize(sent) for sent in reviews_lower]
reviews_token[0]

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


['good', 'but', 'need', 'updates', 'and', 'improvements']

## Task 3. POS tagging using the NLTK pos tagger

Check out the POS tagging chapter from the NLTK book

http://www.nltk.org/book/ch05.html

Examples -

In [5]:
import nltk
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [6]:
# Get the list of tag
nltk.download('tagsets')
nltk.help.brown_tagset()

(: opening parenthesis
    (
): closing parenthesis
    )
*: negator
    not n't
,: comma
    ,
--: dash
    --
.: sentence terminator
    . ? ; ! :
:: colon
    :
ABL: determiner/pronoun, pre-qualifier
    quite such rather
ABN: determiner/pronoun, pre-quantifier
    all half many nary
ABX: determiner/pronoun, double conjunction or pre-quantifier
    both
AP: determiner/pronoun, post-determiner
    many other next more last former little several enough most least only
    very few fewer past same Last latter less single plenty 'nough lesser
    certain various manye next-to-last particular final previous present
    nuf
AP$: determiner/pronoun, post-determiner, genitive
    other's
AP+AP: determiner/pronoun, post-determiner, hyphenated pair
    many-much
AT: article
    the an no a every th' ever' ye
BE: verb 'to be', infinitive or imperative
    be
BED: verb 'to be', past tense, 2nd person singular or all persons plural
    were
BED*: verb 'to be', past tense, 2nd person singular or 

[nltk_data] Downloading package tagsets to /root/nltk_data...
[nltk_data]   Package tagsets is already up-to-date!


In [7]:
nltk.pos_tag(reviews_token[0])

[('good', 'JJ'),
 ('but', 'CC'),
 ('need', 'VBP'),
 ('updates', 'NNS'),
 ('and', 'CC'),
 ('improvements', 'NNS')]

In [8]:
sent = "I like to move it".split()
sent_tagged = nltk.pos_tag(sent)

In [9]:
sent_tagged

[('I', 'PRP'), ('like', 'VBP'), ('to', 'TO'), ('move', 'VB'), ('it', 'PRP')]

In [10]:
# Apply on the dataset
reviews_tagged = [nltk.pos_tag(tokens) for tokens in reviews_token]
reviews_tagged[0]

[('good', 'JJ'),
 ('but', 'CC'),
 ('need', 'VBP'),
 ('updates', 'NNS'),
 ('and', 'CC'),
 ('improvements', 'NNS')]

## Task 4. For the topic model, we would want to include only nouns
 - First, find out all the POS tags that correspond to nouns
 - Limit the data to only terms with these tags


In [11]:
nltk.download('tagsets')
nltk.help.upenn_tagset()

$: dollar
    $ -$ --$ A$ C$ HK$ M$ NZ$ S$ U.S.$ US$
'': closing quotation mark
    ' ''
(: opening parenthesis
    ( [ {
): closing parenthesis
    ) ] }
,: comma
    ,
--: dash
    --
.: sentence terminator
    . ! ?
:: colon or ellipsis
    : ; ...
CC: conjunction, coordinating
    & 'n and both but either et for less minus neither nor or plus so
    therefore times v. versus vs. whether yet
CD: numeral, cardinal
    mid-1890 nine-thirty forty-two one-tenth ten million 0.5 one forty-
    seven 1987 twenty '79 zero two 78-degrees eighty-four IX '60s .025
    fifteen 271,124 dozen quintillion DM2,000 ...
DT: determiner
    all an another any both del each either every half la many much nary
    neither no some such that the them these this those
EX: existential there
    there
FW: foreign word
    gemeinschaft hund ich jeux habeas Haementeria Herr K'ang-si vous
    lutihaw alai je jour objets salutaris fille quibusdam pas trop Monte
    terram fiche oui corporis ...
IN: preposition or

[nltk_data] Downloading package tagsets to /root/nltk_data...
[nltk_data]   Package tagsets is already up-to-date!


You see that for each term, the POS taggger returns a tuple. The first element of the tuple being the term, the second being the tag.

In [12]:
tagged_tuple = nltk.pos_tag(['great'])
tagged_tuple[0]

('great', 'JJ')

In [13]:
pprint(tagged_tuple[0][0])
pprint(tagged_tuple[0][1])

'great'
'JJ'


In [14]:
reviews_noun=[]
for sent in reviews_tagged:
    reviews_noun.append([token for token in sent if re.search("NN.*", token[1])])
reviews_noun[0]

[('updates', 'NNS'), ('improvements', 'NNS')]

In [15]:
# Apply lemmatization
nltk.download('wordnet')

lemm = WordNetLemmatizer()
reviews_lemm=[]

for sent in reviews_noun:
    reviews_lemm.append([lemm.lemmatize(word[0]) for word in sent])

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [16]:
reviews_lemm[0]

['update', 'improvement']

## Task  6. Remove stop words and punctuation (if there are any at all after the POS tagging)

Use NLTK standard stop word list and the punctuations

In [17]:
nltk.download('stopwords')

from string import punctuation
from nltk.corpus import stopwords
stop_nltk = stopwords.words("english")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [18]:
list(punctuation)

['!',
 '"',
 '#',
 '$',
 '%',
 '&',
 "'",
 '(',
 ')',
 '*',
 '+',
 ',',
 '-',
 '.',
 '/',
 ':',
 ';',
 '<',
 '=',
 '>',
 '?',
 '@',
 '[',
 '\\',
 ']',
 '^',
 '_',
 '`',
 '{',
 '|',
 '}',
 '~']

In [19]:
stop_updated = stop_nltk + list(punctuation) + ["..."] + [".."]
reviews_sw_removed=[]
for sent in reviews_lemm:
    reviews_sw_removed.append([term for term in sent if term not in stop_updated])

In [20]:
reviews_sw_removed[1]

['mobile',
 'battery',
 'hell',
 'backup',
 'hour',
 'us',
 'idle',
 'discharged.this',
 'lie',
 'amazon',
 'lenove',
 'battery',
 'charger',
 'hour']

## Task 7. Create a topic model using LDA on the cleaned up data with 10 topics
 - what is the coherence of the model?

 Use gensim for this task

In [21]:
import gensim
import gensim.corpora as corpora
from gensim.models import CoherenceModel
from gensim.models import ldamodel

In [22]:
# Create a Dictionary
id2word = corpora.Dictionary(reviews_sw_removed)
print(len(id2word))

# get the words in Dictionary
print(id2word)

# get the integer id
print(id2word.token2id)

9262
Dictionary<9262 unique tokens: ['improvement', 'update', 'amazon', 'backup', 'battery']...>
{'improvement': 0, 'update': 1, 'amazon': 2, 'backup': 3, 'battery': 4, 'charger': 5, 'discharged.this': 6, 'hell': 7, 'hour': 8, 'idle': 9, 'lenove': 10, 'lie': 11, 'mobile': 12, 'us': 13, 'cash': 14, 'everthey': 15, 'phone': 16, 'problem': 17, 'camerawaste': 18, 'money': 19, 'allot': 20, 'k8': 21, 'reason': 22, 'level': 23, 'ahmedabad': 24, 'hanging': 25, 'lenovo': 26, 'note': 27, 'station': 28, 'year': 29, 'glitch': 30, 'lot': 31, 'option': 32, 'thing': 33, 'wrost': 34, 'damage': 35, 'month': 36, 'item': 37, 'life': 38, 'motherboard': 39, 'battry': 40, 'screen': 41, 'slim': 42, 'headset': 43, 'time': 44, 'card': 45, 'comparison': 46, 'credit': 47, 'deal': 48, 'prize': 49, 'product': 50, 'range': 51, 'seal': 52, 'specification': 53, 'solution': 54, 'smartphone': 55, 'galery': 56, 'speaker': 57, 'camera': 58, 'features.excelent': 59, 'speed.excellent': 60, 'call': 61, 'cast': 62, 'hotspot'

In [23]:
# Convert document (a list of words) into the bag-of-words format
# It contains the word id and its frequency in each document.

texts = reviews_sw_removed
corpus = [id2word.doc2bow(text) for text in texts]
print(corpus[200])

[(36, 1), (143, 1), (314, 1), (415, 1), (416, 1)]


In [24]:
# Apply the LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=10,
                                           passes=10) # Number of passes through the corpus during training.
#  alpha= 0.2,
#  eta= 0.1
#  chunksize=100, #chunksize controls how many documents are processed at a time in the training algorithm

In [25]:
# show the first 10 words belonging to each topic
for idx, topic in lda_model.show_topics(formatted=False,num_words= 10):
  print('Topic: {} \nWords: {}'.format(idx, [w[0] for w in topic]))
  print("\n")

Topic: 0 
Words: ['....', 'money', 'waste', '.....', 'value', 'buy', '......', 'thanks', 'plz', 'dont']


Topic: 1 
Words: ['speaker', 'screen', 'option', 'display', 'processor', 'sound', 'dolby', 'cast', 'music', 'expectation']


Topic: 2 
Words: ['note', 'lenovo', 'k8', 'charger', 'handset', 'everything', 'model', 'piece', 'system', 'please']


Topic: 3 
Words: ['mobile', 'battery', 'performance', 'camera', 'price', 'backup', 'feature', 'range', 'superb', 'mark']


Topic: 4 
Words: ['product', 'amazon', 'delivery', 'service', 'return', 'price', 'customer', 'class', 'policy', 'time']


Topic: 5 
Words: ['battery', 'problem', 'phone', 'issue', 'heating', 'day', 'time', 'hour', 'heat', 'month']


Topic: 6 
Words: ['camera', 'phone', 'battery', 'quality', 'mode', 'feature', 'call', 'price', 'issue', 'ram']


Topic: 7 
Words: ['phone', 'device', 'amazon', 'time', 'day', 'feature', 'month', 'budget', 'care', 'customer']


Topic: 8 
Words: ['camera', 'quality', 'network', 'sim', 'problem', 

In [26]:
# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=reviews_sw_removed, dictionary=id2word, coherence='c_v') #Tokenized texts, needed for coherence models
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Coherence Score:  0.5470073894172673


In [ ]:
!pip install pyLDAvis==3.4.1 # To visualize lda model

In [28]:
# Visualization
import pyLDAvis
import pyLDAvis.gensim

pyLDAvis.enable_notebook()
# vis = pyLDAvis.gensim_models.prepare(lda_model,corpus,id2word)
vis = pyLDAvis.gensim.prepare(lda_model,corpus,id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
6     -0.225043  0.083710       1        1  19.810876
5     -0.193122 -0.154399       2        1  16.868533
7     -0.143578 -0.236344       3        1  12.440987
3     -0.088626  0.178640       4        1   9.141144
8     -0.072999  0.236713       5        1   8.681648
2      0.095223 -0.101220       6        1   8.565363
1      0.048105  0.107899       7        1   8.187492
4      0.106773 -0.154795       8        1   6.201383
9      0.241222 -0.006576       9        1   5.207764
0      0.232045  0.046373      10        1   4.894810, topic_info=         Term         Freq        Total Category  logprob  loglift
16      phone  6695.000000  6695.000000  Default  30.0000  30.0000
50    product  2069.000000  2069.000000  Default  29.0000  29.0000
58     camera  3071.000000  3071.000000  Default  28.0000  28.0000
4     battery  2930.000000  2930.000000  Default  27.0000  27.0000
12     mobile  1413.000000  1413.000000  Default  26.0000  26.0000
..        ...          ...          ...      ...      ...      ...
70   function    45.911888   108.626966  Topic10  -4.5384   2.1558
951       pic    31.750421    72.498590  Topic10  -4.9072   2.1913
647    doesnt    15.797029    25.216074  Topic10  -5.6053   2.5493
218      work    38.544204   196.152745  Topic10  -4.7133   1.3899
114    superb    18.447124   174.751316  Topic10  -5.4502   0.7685

[528 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
143      10  0.997913       ....
103      10  0.997350      .....
499      10  0.992374     ......
367       5  0.974173    .......
2435      4  0.963342  .........
...     ...       ...        ...
349       9  0.989910         ye
1228      3  0.958146  yesterday
2323      5  0.982651       zoom
1292     10  0.968410          👍
6429      4  0.854686          😎

[684 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[7, 6, 8, 4, 9, 3, 2, 5, 10, 1])

## Task 8. Analyze the topics, which pairs of topics can be combined?
 - you can assume that if a pair of topics has very similar top terms, they are very close and can be combined

### Looking at the topics and each terms following can be combined -

** Topic possibly talks about 'pricing'
 Topic closely talks about 'battery related issues'
 Topic talks about 'performance'**

## Task 9. Create topic model using LDA with what you think is the optimal number of topics

 - is the coherence better now?

In [29]:
# Build LDA model
lda_model6 = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=6,
                                           passes=10,
                                           )

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Printing the coherence of the model

In [30]:
# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model6, texts=reviews_sw_removed, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)



Coherence Score:  0.5535012404948645


In [31]:
import pyLDAvis
import pyLDAvis.gensim

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model6,corpus,id2word)
vis

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3     -0.210104  0.040088       1        1  27.279401
2     -0.175885  0.127672       2        1  26.338678
0     -0.150546 -0.099054       3        1  21.690322
1      0.111029 -0.271954       4        1   9.006195
5      0.225658  0.214743       5        1   8.097437
4      0.199848 -0.011496       6        1   7.587966, topic_info=        Term         Freq        Total Category  logprob  loglift
50   product  1946.000000  1946.000000  Default  30.0000  30.0000
4    battery  2874.000000  2874.000000  Default  29.0000  29.0000
12    mobile  1367.000000  1367.000000  Default  28.0000  28.0000
16     phone  6504.000000  6504.000000  Default  27.0000  27.0000
17   problem  1368.000000  1368.000000  Default  26.0000  26.0000
..       ...          ...          ...      ...      ...      ...
260       mi    37.051601    71.813640   Topic6  -5.1912   1.9168
26    lenovo    91.139306   815.377014   Topic6  -4.2911   0.3873
483      box    36.928227   129.315025   Topic6  -5.1946   1.3253
104   charge    35.982553   283.035645   Topic6  -5.2205   0.5161
103    .....    30.227328   242.932494   Topic6  -5.3948   0.4946

[333 rows x 6 columns], token_table=      Topic      Freq   Term
term                        
143       1  0.172420   ....
143       2  0.221441   ....
143       3  0.606850   ....
103       2  0.201702  .....
103       3  0.650386  .....
...     ...       ...    ...
218       4  0.112926   work
218       5  0.295758   work
631       3  0.987711  worth
34        4  0.944213  wrost
349       6  0.990669     ye

[412 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 3, 1, 2, 6, 5])

## Task 10. The business finally needs to be able to interpret the topics
 - name each of the identified topics
 - create a table with the topic name and the top 10 terms in each to present to business

In [32]:
x = lda_model6.show_topics(formatted=False)
topics_words = [(tp[0], [wd[0] for wd in tp[1]]) for tp in x]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [33]:
for topic,words in topics_words:
    print(str(topic)+ "::"+ str(words))
print()

0::['phone', 'camera', 'feature', 'quality', 'note', 'money', '....', 'sound', 'screen', 'music']
1::['product', 'price', 'quality', 'camera', 'range', 'speaker', 'everything', 'delivery', 'feature', 'thanks']
2::['phone', 'time', 'issue', 'lenovo', 'service', 'amazon', 'day', 'problem', 'month', 'call']
3::['battery', 'camera', 'phone', 'issue', 'performance', 'backup', 'day', 'hour', 'quality', 'time']
4::['note', 'k8', 'hai', 'h', 'lenovo', 'ho', 'excellent', 'smartphone', 'class', 'cell']
5::['mobile', 'problem', 'network', 'heating', 'sim', 'jio', 'volta', 'heat', 'super', 'plz']



/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [34]:
#possible topics from terms present

#Topic1 = product accessories
#Topic2 = amazon
#Topic3 = pricing
#Topic4 = phone performance
#Topic5 = battery related issues
#Topic6 = camera quality
#Topic7 = sound features
#Topic8 = overall general phone features

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
